In [175]:
import argparse
import math
import os
import re
import unicodedata
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytz
import seaborn as sns
from openpyxl import load_workbook

In [176]:
file_path = 'report1728052678490.csv'

report_df = pd.read_csv(file_path, encoding='Latin-1')

In [177]:
# Column renaming mapping
column_renaming_map = {
    "case_owner": "owner",
    "case_owner_alias": "owner_alias",
    "case_owner_role": "owner_role",
    "owner_is_queue": "is_queue",
    "case_origin": "origin",
    "case_number": "case_number",
    "case_reason": "reason",
    "status": "status",
    "open": "is_open",
    "subject": "subject",
    "description": "description",
    "account_name": "account",
    "account_record_type": "record_type",
    "account_tier_name": "tier",
    "parent_account": "parent_account",
    "region": "region",
    "datetime_opened": "created_at",
    "time_to_in_progress_hours": "response_hrs",
    "age_hours": "age_hrs",
}

# Tier mapping
tier_mapping = {
    "Diamond": "Diamond",
    "Platinum": "Platinum",
    "Gold": "Gold",
    "Silver": "Silver",
    "Bronze": "Bronze",
    "Not Tiered": "Not Tiered",
    "Unsupported": "Not Tiered",
}

# Region mapping
region_mapping = {
    "ASIA PACIFIC": "APAC",
    "EMEA": "EMEA",
    "NORTH AMERICA": "AMER",
    "LATAM": "AMER",
}

# Allowed regions
allowed_regions = ["APAC", "EMEA"]

# Data types for columns
column_data_types = {
    "owner": "string",
    "owner_alias": "string",
    "owner_role": "string",
    "is_queue": "bool",
    "origin": "string",
    "case_number": "string",
    "reason": "string",
    "status": "string",
    "is_open": "bool",
    "subject": "string",
    "description": "string",
    "account": "string",
    "record_type": "string",
    "tier": "string",
    "parent_account": "string",
    "region": "string",
    "created_at": "datetime64[ns, America/Mexico_City]",
    "response_hrs": "float",
    "age_hrs": "float",
}

# Owner team mapping
owner_team_mapping = {
    "CTAM": {
        "Team USA": [
            "apersaud", "asachdev", "asangkar", "biron", "gdelucen", "ghudson", "hpatel2", "jaguirre", "jcescoba", "jguzm",
            "kbowdenf", "kcooper2", "kla", "kquan", "lbyrom", "mdardign", "mwalton2", "pfullmer", "scastell", "yjkim",
            'lugalde', 'kla', 'gdelucen', 'jguzm', 'mwalton2', 'kbowdenf', 'kcooper2', 'kquan', 'mdardign', 'jaguirre', 'scastell','sschutz'
        ],
        "Team Enroute": [
            "aegorova", "aferrara", "agarduno", "amoreno", "apena", "aperez", "arodrigu", "bsussela", "bvazquez", "cbarbosa",
            "crevilla", "dsalas", "eguzman", "esolis", "fdelapen", "gdiaz", "gmalagon", "janovero", "jgranado", "jromopad",
            "kalvarez", "lugalde", "mdeleon", "mmendez", "molais", "msilva", "nroldan", "pmadero", "rbravope", "reliasfe",
            "rflores", "rgarcia", "rvazquez", "samaradh", "scano", "sgonzale", "vperez", "yegraf", 'amoreno', 'dsalas',
            'nroldan', 'amartos', 'sgonzale', 'cbarbosa', 'akate', 'bperraut', 'hpatel2', 'fmuehlba', 'abentahi', 'agarduno', 'nicwong', 'apena', 'rflores',
            'aferrara', 'jbahn', 'kmakode', 'jcrow', 'yjkim', 'msalami', 'rfurlan', 'yegraf', 'scano', 'fmulas', 'jgranado', 'rvazquez', 'skeskar'
        ],
        "Team APAC": [
            "abentiga", "jbahn", "jng", "kpak", "manan", "skapoor", "tvalerio", "vtiama", 'manan', 'tvalerio'
        ],
        "Team EMEA": [
            "abentahi", "amartos", "bperraut", "cadams", "cmackenz", "crisc", "cyu", "ebine", "fmuehlba", "fmulas",
            "gsmit", "hmasih", "jcrow", "jvoigtsb", "kforson", "mbendada", "mbespalo", "mcowley", "msalami", "mtipa",
            "nicwong", "pball", "pbenoit", "pprin", "rde l", "rfurlan", "rwajahat", "steo", "tbillimo", "vbuschwa",
            'amartos', 'cadams', 'ebine', 'jvoigtsb', 'pprin', 'steo',
            'rwajahat', 'mtipa', 'abentahi', 'bperraut', 'fmuehlba', 'abentahi',
            'nicwong', 'rfurlan'
        ],
        "Team PUNE": [
            "akate", "anaiknaw", "anair", "asadar", "asharma2", "athombre", "awankhad", "braghuva", "dtare", "ikahmed",
            "kmakode", "mthakur", "nkaul", "pdeshmuk", "rjadhawa", "sdutta", "skeskar", "sranjane", "vbhedasg",
            'akate', 'pdeshmuk', 'skeskar'
        ],
        "Queue": [
            "APAC Collections", "Austrailia/New Zealand - Tier 1", "Southeast Asia - Tier 1", "TAM APAC", "CS US Region Support",
            "TAM USA", "USA - Tier 1", "USA Buy Side", "Closed Cases Queue", "Central Europe - Tier 1", "EMEA Buy Side",
            "EMEA Managers", "EMEA Publisher/Platform", "France & Belgium - Tier 1", "Italy - Tier 1", "Northern Europe - Tier 1",
            "Spain & Portugal - Tier 1", "TAM EMEA", "India - Tier 1", "Theorem CS Queue", "APAC Collections", "Platform TAM queue",
            "Product Specialists - Walled Gardens"
        ],
        "IAS Bot": ["jinte"],
        "Unassigned": []
    },
    "CSO": {
        "Team CSO": ["Customer Success Operations", "rronghan", "ddevkate", 'asingh2', 'sdube']
    },
    "PTAM": {
        "Team PTAM": ["dwhite", "mbaudist", "arosasdi", 'asangkar']
    },
    "Other": []
}

# Timezone settings
original_timezone = "America/Mexico_City"
timezone_mapping = {
    "APAC": "Asia/Tokyo",
    "EMEA": "Europe/London",
    "AMER": "America/New_York",
}

# Business hours settings
business_hours_per_day = 9  # 8 AM to 5 PM
business_days_per_week = 5
business_start_hour = 9
business_end_hour = 17

# Launch specific cases
launch_specific_cases = [
    "Unmapped Publisher",
    "Tagging",
    "Tag Wrapping Request",
    "Innovid Error",
    "New API Tag Wrapping Pending Request",
]

In [178]:
# Clean Column Names
def clean_column_name(column_name):
    """
    Clean and normalize a column name by:
    - Removing accents and diacritics.
    - Removing non-alphanumeric characters.
    - Replacing whitespace with underscores.
    - Converting to lowercase.
    """
    # Remove accents and diacritics
    normalized_name = ''.join(
        (char for char in unicodedata.normalize('NFD', column_name) if unicodedata.category(char) != 'Mn')
    )
    # Remove non-alphanumeric characters
    alphanumeric_name = re.sub(r'[^\w\s]', '', normalized_name)
    # Replace whitespace with underscores and convert to lowercase
    cleaned_name = re.sub(r'\s+', '_', alphanumeric_name.strip().lower())
    return cleaned_name

report_df.columns = [clean_column_name(column) for column in report_df.columns]

In [179]:
# Rename columns for better readability and consistency
report_df.rename(columns=column_renaming_map, inplace=True)

In [180]:
# Handle Missing Values
# Fill missing values for 'owner_role' where 'is_queue' is True
report_df.loc[(report_df['owner_role'].isna()) & (report_df['is_queue'] == True), 'owner_role'] = 'Queue'

# Fill missing values for specific columns
report_df['origin'] = report_df['origin'].fillna('Integral Platform')
report_df['reason'] = report_df['reason'].fillna('Other')
report_df['subject'] = report_df['subject'].fillna('No Subject Provided')
report_df['account'] = report_df['account'].fillna('Unknown Account')
report_df['record_type'] = report_df['record_type'].fillna('Buy')
report_df['parent_account'] = report_df['parent_account'].fillna('Standalone Account')

# Fill missing 'description' with 'subject' if 'description' is NaN
report_df['description'] = report_df.apply(
    lambda row: row['subject'] if pd.isna(row['description']) else row['description'], axis=1
)

# Fill missing 'response_hrs' with 'age_hrs' if 'response_hrs' is NaN
report_df['response_hrs'] = report_df.apply(
    lambda row: row['age_hrs'] if pd.isna(row['response_hrs']) else row['response_hrs'], axis=1
)

In [181]:
# Replace correct Values
# Replace correct values in 'owner_role' column using regex
report_df['owner_role'] = report_df['owner_role'].str.replace(
    r'^Technical Account Management.*', 'CTAM', regex=True
)
report_df['owner_role'] = report_df['owner_role'].str.replace(
    r'^Platform Technical.*', 'PTAM', regex=True
)
report_df['owner_role'] = report_df['owner_role'].str.replace(
    r'^Customer.*', 'CSO', regex=True
)

In [182]:
# Map Values
# Replace values in 'tier' column using tier mapping
report_df['tier'] = report_df['tier'].replace(tier_mapping)

# Replace values in 'region' column using region mapping
report_df['region'] = report_df['region'].replace(region_mapping)

# Apply conditional replacement for 'region' column
report_df['region'] = report_df['region'].apply(
    lambda region: 'AMER' if region not in allowed_regions else region
)

In [183]:
# Convert Data Types

In [200]:
# Handle duplicates
# Find Unmapped Publishers with the exact same description
unmapped_publishers_duplicates_mask = report_df[report_df['reason'] == 'Unmapped Publisher']
unmapped_publishers_duplicates_mask



,owner,owner_alias,owner_role,is_queue,origin,case_number,reason,status,is_open,subject,description,account,record_type,tier,parent_account,region,created_at,response_hrs,age_hrs,owner_team


In [185]:
# Map Owner Alias to Team
def get_team(row):
    owner_role = row['owner_role']
    owner_alias = row['owner_alias']
    
    if owner_role in owner_team_mapping:
        for team, members in owner_team_mapping[owner_role].items():
            if owner_alias in members:
                return team
    return 'No Team'

report_df['owner_team'] = report_df.apply(get_team, axis=1)

In [186]:
# Filter owners without a team and drop duplicates
owners_without_team_df = report_df[report_df['owner_team'] == 'No Team'][['owner', 'owner_alias', 'owner_role', 'owner_team']].drop_duplicates()

# Group by owner_role and count the number of unique owners without a team
grouped_owners = owners_without_team_df.groupby('owner_role').size().reset_index(name='count')
print("Number of unique owners without a team by role:")
print(grouped_owners)

# Display the dataframe
print("\nOwners without a team:")
print(owners_without_team_df)

# Add owners to the owner_team_mapping
for index, row in owners_without_team_df.iterrows():
    owner_role = row['owner_role']
    owner_alias = row['owner_alias']
    if owner_role in owner_team_mapping:
        if isinstance(owner_team_mapping[owner_role], dict):
            if 'Unassigned' not in owner_team_mapping[owner_role] or not isinstance(owner_team_mapping[owner_role]['Unassigned'], list):
                owner_team_mapping[owner_role]['Unassigned'] = []
            owner_team_mapping[owner_role]['Unassigned'].append(owner_alias)
        else:
            owner_team_mapping[owner_role].append(owner_alias)
    elif owner_role == 'CTAM':
        if 'Unassigned CTAM' not in owner_team_mapping['CTAM']:
            owner_team_mapping['CTAM']['Unassigned CTAM'] = []
        owner_team_mapping['CTAM']['Unassigned CTAM'].append(owner_alias)
        print(f"Owner '{owner_alias}' with role 'CTAM' has been added to 'Unassigned CTAM'. Please manually assign them to the correct group.")
    else:
        print(f"Warning: '{owner_alias}' with role '{owner_role}' is not recognized. Please manually add to the owner_team_mapping.")

# Function to remove duplicates from lists in the dictionary
def remove_duplicates_from_mapping(mapping):
    for key, value in mapping.items():
        if isinstance(value, dict):
            remove_duplicates_from_mapping(value)
        elif isinstance(value, list):
            mapping[key] = list(set(value))

# Remove duplicates from owner_team_mapping
remove_duplicates_from_mapping(owner_team_mapping)

# Print the updated owner_team_mapping for 'Unassigned CTAM'
print("\nUpdated owner_team_mapping (only 'Unassigned CTAM'):")
print(owner_team_mapping['CTAM'].get('Unassigned CTAM', []))

# Instructions for manually assigning CTAM members
print("\nTo manually assign 'Unassigned CTAM' members to their corresponding groups, follow these steps:")
print("1. Identify the correct group for each member in 'Unassigned CTAM'.")
print("2. Move the member from 'Unassigned CTAM' to the appropriate group in 'owner_team_mapping['CTAM']'.")
print("3. Remove the member from 'Unassigned CTAM' once they have been assigned to a group.")


Number of unique owners without a team by role:
  owner_role  count
0       PTAM      2
1      Queue     20

Owners without a team:
                                      owner  \
25                       Closed Cases Queue   
38                             USA - Tier 1   
70                France & Belgium - Tier 1   
246                 Central Europe - Tier 1   
648                Northern Europe - Tier 1   
769                      Margaret Bespalova   
866             Customer Success Operations   
2564                           USA Buy Side   
8639                           Grace Hudson   
11964                              TAM EMEA   
14662       Austrailia/New Zealand - Tier 1   
22992                      Theorem CS Queue   
27500                  CS US Region Support   
28508  Product Specialists - Walled Gardens   
31596                         EMEA Buy Side   
31766               Southeast Asia - Tier 1   
35384                               TAM USA   
69713                 